In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)

In [2]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [3]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_fill.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_fill.csv')
testData = pd.read_csv(testPath)
submPath = os.path.join(homePath, 'submission.csv')
submData = pd.read_csv(submPath)

In [4]:
trainData['acc_now_delinq'] = list(map(int, trainData['acc_now_delinq']))
trainLabel = trainData['acc_now_delinq']
del trainData['acc_now_delinq']

In [5]:
trainData['annual_inc'].fillna(trainData['annual_inc'].mean(), inplace=True)
trainData['emp_title'].fillna(0, inplace=True)
testData['emp_title'].fillna(0, inplace=True)

In [13]:
testData.drop("acc_now_delinq", axis=1, inplace=True)

In [62]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=110, min_samples_split=90, min_samples_leaf=15,max_depth=10,oob_score=True,max_features=10)

from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.05,
                     n_estimators=1000,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     reg_alpha=0.001,
                     reg_lambda=0.001,
                     scale_pos_weight=1)

import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
                                 max_features='sqrt', subsample=0.7)


from mlxtend.classifier import StackingClassifier 
stack_clf = StackingClassifier(classifiers=[gbdt, lgb_clf, rnd_clf, xgb], meta_classifier=xgb)


In [66]:
stack_clf.fit(trainData, trainLabel)

StackingClassifier(average_probas=False,
          classifiers=[GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=7,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min...eg_alpha=0.001, reg_lambda=0.001,
       scale_pos_weight=1, seed=None, silent=True, subsample=0.8)],
          meta_classifier=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0.001, reg_lambda=0.001,
       scale_pos_weight=1, seed=None, silent=True, subsample=0.8),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=Fa

In [67]:
stack_pred = stack_clf.predict(testData)

In [68]:
stack_pred.sum()

244